# How to build a simple Agent LLM App with LangGraph
* Very basic tool-using Agent LLM App with memory.

Language models can't do anything on their own; they can only create text based on what you ask them. However, LangChain allows people to build agents—think of these as smart systems or helpers— that use language models to think and decide what to do next.

Here’s how it works:
1. **Use the Language Model as a Brain**: The agent uses the language model to figure out which actions it should take, based on the information it has or what you ask it to do.
2. **Taking Action**: After deciding, the agent then goes ahead and does those actions.
3. **Learning and Adjusting**: Once the actions are done, the results can be given back to the language model. This helps the model check if everything is complete or if it needs to do something else.

So, essentially, LangChain helps turn a language model from just a tool for writing and answering into a system that can act and react, almost like a very simple robot brain.

Here we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it calling the search tool, and have conversations with it.

## Concepts included
* [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph).

LangGraph is a library created by the LangChain team for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-simple-agent.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-simple-agent**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [1]:
#!pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Agents
* Agents use LLMs as reasoning engines to determine which actions to take.

## Tool-using Agent
* For this basic agent we will use just one tool. In next advanced projects, you will learn how to use agents with several tools.
* **Our tool of choice will be Tavily** - a search engine. 
#### Tavily API Key
* You will need to add your Tavily API key in the .env file. Obtain a key by signing up on their [website](https://tavily.com/).

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [6]:
#pip install langgraph

#### Define tools

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [7]:
#pip install langchain-community

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
print(search.invoke("Who are the top stars of the 2024 Eurocup?"))

[{'url': 'https://www.givemesport.com/ranking-best-players-euro-2024/', 'content': "Bukayo Saka 3. Jude Bellingham The best player at Euro 2024 is... GiveMeSport. Menu ... make the top 20 players at this summer's European Championships. ... 3-0 victory and is one of several stars"}, {'url': 'https://www.90min.com/features/10-best-players-euro-2024-ranked', 'content': "The 10 best players of Euro 2024 - ranked The 10 best players of Euro 2024 - ranked The best players of Euro 2024 The Three Lions number one was once again in superb form for his nation throughout Euro 2024, particularly in the early rounds when England's attackers were struggling for form at the other end of the pitch. There was point during Euro 2024 in which Jamal Musiala looked certain to win Player of the Tournament. As expected, he was bloody brilliant at Euro 2024 for Spain and picked up the Player of the Tournament award, but he's not our winner..."}]


In [7]:
tools = [search]

* **In order to enable this model to do tool calling we use .bind_tools** to give the language model knowledge of these tools:

In [8]:
llm_with_tools = llm.bind_tools(tools)

## Create the agent
* We will be using LangGraph to construct the agent. 
* **Note that below we are passing in the origina chat model, not the llm_with_tools we built later**. That is because create_tool_calling_executor will call .bind_tools for us under the hood.

In [11]:
# from langgraph.prebuilt import chat_agent_executor

# agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

* In the latest version of langgraph, we have a most modern way to do this:

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

## Run the agent
* Let's first try it with .invoke():

In [10]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', id='9bc9679f-3b64-4c58-8de4-e52329228a7f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_66FnFc2tVWUr8IHYFWX0mUky', 'function': {'arguments': '{"query":"Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 90, 'total_tokens': 113, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2c0f25a4-da1f-498f-b3ea-72a48065d0e3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Eurocup 2024 location'}, 'id': 'call_66FnFc2tVWUr8IHYFWX0mUky', 'type': 'tool_call'}], usage_metadata={'input_to

* Now let's try it with .stream():

In [11]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and where will it be the 2024 Eurocup final match?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BKA2uBp7exTPl4GWHFG8lij3', 'function': {'arguments': '{"query":"2024 Eurocup final match date and location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 94, 'total_tokens': 120, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1604f822-b1ee-423a-af3e-3d673b04953e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup final match date and location'}, 'id': 'call_BKA2uBp7exTPl4GWHFG8lij3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 94, 'output_tokens': 26, 'total_tokens

## Adding memory
* Adding memory in LangGraph is very similar to what we did with LangChain.

In [12]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

* Let's create our new agent with memory and set one thread_id so the agent can create a memory for each session as we did with our previous conversational RAG app:

In [13]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "001"}}

* Let's now try this new agent with .stream():

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bZDJoeeACTs1s6HTiICpPaw4', 'function': {'arguments': '{"query":"2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 89, 'total_tokens': 112, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-02da9e92-701e-41bb-9c24-b2d56b7360eb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_bZDJoeeACTs1s6HTiICpPaw4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 89, 'output_tokens': 23, 'total_tokens': 112})]}}
----
{'tools': {'mes

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MfIKtLfSBdIMiJI9H4QwlK3b', 'function': {'arguments': '{"query":"top players Spain Euro 2024 team"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 442, 'total_tokens': 467, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-34b890b0-2a3f-43ee-9da4-be74220d8402-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'top players Spain Euro 2024 team'}, 'id': 'call_MfIKtLfSBdIMiJI9H4QwlK3b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 442, 'output_tokens': 25, 'total_tokens': 467})]}}
----
{

* Let's change the thread_id and see what happens:

In [16]:
config = {"configurable": {"thread_id": "002"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Could you please provide more context or specify which soccer team you're referring to? There are many teams around the world, and I want to ensure I give you the correct information.", response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 87, 'total_tokens': 123, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba6c22af-67f1-4aed-adf8-92b90e3436df-0', usage_metadata={'input_tokens': 87, 'output_tokens': 36, 'total_tokens': 123})]}}
----


* As you can see, when we changed the thread_id we started a different conversation memory, so our app does not remember the previous interaction and instead of giving us the right answer it decided to hallucinate.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-simple-agent.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-simple-agent.py